In [6]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Activation, Dense, Dropout, RNN
from tensorflow.keras import Model
from keras_model import build_model

In [7]:
df = pd.read_csv('..\\data\\processed2\\processed_data.csv', index_col=0)

In [8]:
df['event_final'] = df['event_final'].astype('category')

In [9]:
df['event_final_code'] = df['event_final'].cat.codes

In [10]:
first = [0]*len(df)
second = [0]*len(df)
third = [0]*len(df)
for i in df.index:
    if isinstance(df.loc[i, '1st runner'], str):
        first[i] = 1
    if isinstance(df.loc[i, '2nd runner'], str):
        second[i] = 1
    if isinstance(df.loc[i, '3rd runner'], str):
        third[i] = 1
    if i % 1000 == 0:
        print(i)
first = np.array(first)
second = np.array(second)
third = np.array(third)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [11]:
df['first'] = first
df['second'] = second
df['third'] = third

In [98]:
situation = df[['inning', 'batting team', 'outs', 'visiting_score', 'home_score', 'first', 'second', 'third']].values
batter = df['res batter'].astype('category').cat.codes.values
pitcher = df['res pitcher'].astype('category').cat.codes.values
balls = df['balls'].values
strikes = df['strikes'].values
fouls = df['fouls'].values
outcome = df['event_final_code'].values
outcome_onehot = pd.get_dummies(df['event_final_code']).values

In [99]:
situation[:, 0] -= 1

In [100]:
class AtBatCell(tf.keras.layers.Layer):

    def __init__(self, n_batters, n_pitchers, situation_size, states):
        super(AtBatCell, self).__init__()
        self.n_batters = n_batters
        self.n_pitchers = n_pitchers
        self.situation_size = situation_size
        self.states = states
        self.state_size = tf.TensorShape([self.n_batters + self.n_pitchers, states])

    def build( self, input_shape ):
        self.Wz = self.add_weight('input z',
                                    shape=[self.states*2, self.situation_size])
        self.Wr = self.add_weight('input r',
                                    shape=[self.states*2, self.situation_size])
        self.Wh = self.add_weight('input h',
                                    shape=[self.states*2, self.situation_size])
        self.Uz = self.add_weight('state z',
                                    shape=[self.states*2, self.states*2])
        self.Ur = self.add_weight('state r',
                                    shape=[self.states*2, self.states*2])
        self.Uh = self.add_weight('state h',
                                    shape=[self.states*2, self.states*2])
        self.bz = self.add_weight('const z',
                                    shape=[self.states*2, 1])
        self.br = self.add_weight('const r',
                                    shape=[self.states*2, 1])
        self.bh = self.add_weight('const h',
                                    shape=[self.states*2, 1])
        super(AtBatCell, self).build(input_shape)
    
    def call(self, inputs, state):
        x, b, p = inputs
        x = tf.reshape(x, (-1, 1))
        state = state[0] + tf.zeros((state[0].shape))
        h = self.get_states(state, b, p)
        hp = self.GRU(x, h)
        state = self.update_states(state, b, p, hp, h)
        state = tf.reshape(state, (1, state.shape[0], state.shape[1]))
        return state, [state]

    def GRU(self, x, h):
        z = self.Wz @ x + self.Uz @ h + self.bz
        z = tf.math.sigmoid(z)
        r = self.Wr @ x + self.Ur @ h + self.br
        r = tf.math.sigmoid(r)
        m = self.Wh @ x + self.Uh @ (r * h) + self.bh
        m = tf.math.tanh(m)
        hp = z * h + (1-z) * m
        return hp

    def get_states(self, states, batter, pitcher):
        state_batter = tf.gather_nd(states[0], batter)
        state_pitcher = tf.gather_nd(states[0], pitcher)
        h = tf.concat((state_batter, state_pitcher), axis=0)
        return tf.reshape(h, (-1, 1))
    
    def update_states(self, states, batter, pitcher, hp, h):
        dh = hp - h
        dh = tf.reshape(dh, (2, -1))
        indices = tf.reshape([batter, pitcher], (2, 1))
        states = states[0] + tf.scatter_nd(indices, dh, states[0].shape)
        return states

class AtBatRNN(RNN):
    
    def __init__(self, n_batters, n_pitchers, situation_size, states, 
                 return_sequences=True, return_state=False,
                 stateful=True, unroll=False):
        self.n_batters = n_batters
        self.n_pitchers = n_pitchers
        self.situation_size = situation_size
        self.states = states
        cell = AtBatCell(n_batters, n_pitchers, situation_size, states)
        super(AtBatRNN, self).__init__(cell, 
            return_sequences=return_sequences, return_state=return_state,
            stateful=stateful, unroll=unroll)
        
    def call(self, inputs, initial_state=None, constants=None):
        return super(AtBatRNN, self).call(inputs, initial_state=initial_state, constants=constants)

In [142]:
class PredictionLayer(tf.keras.layers.Layer):

    def __init__(self, n_batters, n_pitchers, situation_size, states, classes):
        super(PredictionLayer, self).__init__()
        self.n_batters = n_batters
        self.n_pitchers = n_pitchers
        self.situation_size = situation_size
        self.states = states
        self.classes = classes

    def build( self, input_shape ):
        self.W = self.add_weight('input',
                                   shape=[self.classes, self.situation_size])
        self.U = self.add_weight('state',
                                   shape=[self.classes, self.states*2])
        self.b = self.add_weight('const',
                                   shape=[self.classes, 1])
        super(PredictionLayer, self).build(input_shape)

    def call(self, inputs):
        xp, states, bp, pp = inputs
        xp = tf.expand_dims(xp, axis=-1)
        hp = self.get_states(states, bp, pp)
        return self.W @ xp + self.U @ hp + self.b

    def get_states(self, states, batters, pitchers):
        seq_len = batters.shape[1]
        state_batters = tf.gather_nd(states[0], batters)
        state_pitchers = tf.gather_nd(states[0], pitchers)
        h = tf.concat((state_batters, state_pitchers), axis=1)
        return tf.reshape(h, (seq_len, -1, 1))

In [143]:
def build_model( n_batters, n_pitchers, situation_size, states, output_shape, seq_len ):
    
    '''
    This function builds the tensorflow model.
    '''
    b = Input(batch_shape=(1, seq_len, 1), dtype=tf.int32)
    p = Input(batch_shape=(1, seq_len, 1), dtype=tf.int32)
    x = Input(batch_shape=(1, seq_len, situation_size))
    
    h_p = AtBatRNN(n_batters, n_pitchers, situation_size, states)(tuple([x, b, p]))
    
    b_p = Input(batch_shape=(None, seq_len, 1), dtype=tf.int32)
    p_p = Input(batch_shape=(None, seq_len, 1), dtype=tf.int32)
    x_p = Input(batch_shape=(None, seq_len, situation_size))
    
    result = PredictionLayer(n_batters, n_pitchers, situation_size, states, output_shape)([x_p, h_p, b_p, p_p])
    
    result = Activation('softmax')(result)

    pitch_count = PredictionLayer(n_batters, n_pitchers, situation_size, states, output_shape)([x_p, h_p, b_p, p_p])
    
    model = Model(inputs=[b, p, x, b_p, p_p, x_p],
                  outputs=[result, pitch_count])
    
    return model

In [144]:
train = range(840000)
test = range(840000, len(df))

In [145]:
seq_len = 2000

In [146]:
seq_start = [len(train)%seq_len + seq_len*i for i in range(len(train)//seq_len)]
time_series = [range(start, start+seq_len) for start in seq_start]
time_series_plus = [range(start+1, start+1+seq_len) for start in seq_start]
time_series_test = [range(len(test))]
time_series_test_plus = [range(1,len(test)+1)]

In [147]:
def convert_to_time_series(tensor, time_series_list):
    tensor_list = []
    for r in time_series_list:
        tensor_list += [tensor[r]]
    return np.stack(tensor_list)

In [148]:
batter = np.expand_dims(batter, axis=1).astype(np.float32)
pitcher = np.expand_dims(pitcher, axis=1).astype(np.float32)

In [149]:
situation_batched = convert_to_time_series(situation, time_series)
batters_batched = convert_to_time_series(batter, time_series)
pitchers_batched = convert_to_time_series(pitcher, time_series)
situation_batched_plus = convert_to_time_series(situation, time_series_plus)
batters_batched_plus = convert_to_time_series(batter, time_series_plus)
pitchers_batched_plus = convert_to_time_series(pitcher, time_series_plus)

In [150]:
pitch_count = np.concatenate(
    (np.expand_dims(balls, axis=1), np.expand_dims(strikes, axis=1), np.expand_dims(fouls, axis=1)), axis=1
)

In [151]:
pitch_count_batched = convert_to_time_series(pitch_count, time_series_plus)
outcome_onehot_batched = convert_to_time_series(outcome_onehot, time_series_plus)

In [152]:
n_batters = np.unique(batter).shape[0]
n_pitchers = np.unique(pitcher).shape[0]
situation_size = situation.shape[1]
states = 8
classes = outcome_onehot.shape[1]

In [153]:
model = build_model( n_batters, n_pitchers, situation_size, states, classes, seq_len )

In [154]:
def compile_model(model, LR):
    adam = tf.keras.optimizers.Adam(lr=LR)
    model.compile(
        optimizer=adam,
        loss=['categorical_crossentropy', 'mean_squared_error'],
        metrics=['accuracy', 'RootMeanSquaredError'])
    return model

In [155]:
model = compile_model(model, 1e-3)

In [156]:
for i in range(200):
    print('epoch: '+str(i+1))
    model.fit(x=[batters_batched, pitchers_batched, situation_batched,
                 batters_batched_plus, pitchers_batched_plus, situation_batched_plus],
              y=[pitch_count_batched, outcome_onehot_batched], 
              batch_size=1,
              epochs=1,
              verbose=1)

epoch: 1


ValueError: Error when checking input: expected input_31 to have 3 dimensions, but got array with shape (420, 2000, 1, 1, 1)